In [1]:
import numpy as np
from prototypes.utility.data import DataLoader
from prototypes.utility.data import ProjectConfiguration
import matplotlib.pyplot as plt
from prototypes.classical.segmentation.transformers import BlackBarsRemover, OtsuThresholdingSegmentation
from prototypes.classical.descriptors.texture import LBPTransformer, GaborTransformer, HoGTransformer
from prototypes.classical.descriptors.vetorizer import PCAVectorizer
import cv2

%load_ext autoreload
%autoreload 2

In [2]:
config = ProjectConfiguration("../config.json")

In [3]:
config.get_keys()

In [4]:
data_loader = DataLoader(data_path=config.get_value("TRAIN_IMAGES_PATH"), metadata_path=config.get_value("TRAIN_METADATA"))

In [5]:
cancer_images = data_loader.get_data(target=1, n_sample=300, width=128, height=128)
non_cancer_images = data_loader.get_data(target=0, n_sample=300, width=128, height=128)
otsu_thresholding = OtsuThresholdingSegmentation()
lbp_transformer = LBPTransformer(p=8, r=1)

In [6]:
cancer_images.shape

In [7]:
_, ax = plt.subplots(1, 4, figsize=(15, 15))

gray_scaled = cv2.cvtColor(cancer_images[4], cv2.COLOR_RGB2GRAY)
thresholded = otsu_thresholding.transform(gray_scaled)
lbp_thresholded = lbp_transformer.transform([thresholded])[0]
lbp_wholeimage = lbp_transformer.transform([gray_scaled])[0]

ax[0].imshow(cancer_images[4])
ax[1].imshow(thresholded, "gray")
ax[2].imshow(lbp_thresholded, "gray")
ax[3].imshow(lbp_wholeimage, "gray")

In [8]:
gray_scaled.mean(), gray_scaled.std()

In [9]:
lbp_thresholded.mean(), lbp_thresholded.std()

In [10]:
lbp_wholeimage.mean(), lbp_wholeimage.std()

In [11]:
_, ax = plt.subplots(1, 4, figsize=(15, 15))

gray_scaled = cv2.cvtColor(non_cancer_images[4], cv2.COLOR_RGB2GRAY)
thresholded = otsu_thresholding.transform(gray_scaled)
lbp_thresholded = lbp_transformer.transform([thresholded])[0]
lbp_wholeimage = lbp_transformer.transform([gray_scaled])[0]

ax[0].imshow(non_cancer_images[4])
ax[1].imshow(thresholded, "gray")
ax[2].imshow(lbp_thresholded, "gray")
ax[3].imshow(lbp_wholeimage, "gray")

In [12]:
gray_scaled.mean(), gray_scaled.std()

In [13]:
lbp_thresholded.mean(), lbp_thresholded.std()

In [14]:
lbp_wholeimage.mean(), lbp_wholeimage.std()

# Distribution test

Pixel intesity in gray scale

In [8]:
bening_images = data_loader.get_data(target=0, width=64, height=64, mode="gray")

In [9]:
malign_images = data_loader.get_data(target=1, width=64, height=64, mode="gray")

In [17]:
lbp_benign = lbp_transformer.transform(bening_images)
lbp_malign = lbp_transformer.transform(malign_images)

lbp_thresholded_benign = otsu_thresholding.transform(lbp_transformer.transform(bening_images))
lbp_thresholded_malignant = otsu_thresholding.transform(lbp_transformer.transform(malign_images))

lbp_benign_mean, lbp_benign_std = np.mean(lbp_benign), np.std(lbp_benign)
lbp_malign_mean, lbp_malign_std = np.mean(lbp_malign), np.std(lbp_malign)

lbp_thresholded_benign_mean, lbp_thresholded_std = np.mean(lbp_thresholded_benign), np.std(lbp_thresholded_benign)
lbp_thresholded_malignant_mean, lbp_thresholded_malignant_std = np.mean(lbp_thresholded_malignant), np.std(lbp_thresholded_malignant)

bening_mean, bening_std = bening_images.mean(), bening_images.std()
malignant_mean, malignant_std = malign_images.mean(), malign_images.std()

In [81]:
x = np.linspace(-255, 256, 256)

def gaussian_dist(x, mean, std):
    return (1/(std*np.sqrt(2*np.pi))) * np.exp(-(x-mean)**2/(2*std**2))

In [19]:
plt.plot(x, gaussian_dist(x, bening_mean, bening_std), label="Benign")
plt.plot(x, gaussian_dist(x, malignant_mean, malignant_std), label="Malignant")

plt.plot(x, gaussian_dist(x, lbp_benign_mean, lbp_benign_std), label="LBP-Benign")
plt.plot(x, gaussian_dist(x, lbp_malign_mean, lbp_malign_std), label="LBP-Malignant")

plt.plot(x, gaussian_dist(x, lbp_thresholded_benign_mean, lbp_thresholded_std), label="LBP-Thresholded-Benign")
plt.plot(x, gaussian_dist(x, lbp_thresholded_malignant_mean, lbp_thresholded_malignant_std), label="LBP-Thresholded-Malignant")

plt.legend()
plt.show()

# Distribution based on gabor banks

In [58]:
import numpy as np

In [59]:
gabor_transformer = GaborTransformer(frequency=1/100, theta=np.pi/4, sigma_x=5, sigma_y=5)

In [91]:
gabor_benign = gabor_transformer.transform(bening_images)

In [92]:
from prototypes.classical.descriptors.vetorizer import PCAVectorizer

In [93]:
pca_vectorizer_b = PCAVectorizer(components=10)
pca_vectorizer_m = PCAVectorizer(components=10)

In [94]:
plt.imshow(gabor_benign[0], "gray")

In [95]:
plt.imshow(gabor_benign[-1], "gray")

In [96]:
gabor_malignant = gabor_transformer.transform(malign_images)

In [ ]:
pca_vectorizer_b.fit(gabor_benign)
pca_vectorizer_m.fit(gabor_malignant)

# mean and std for each component of the PCA to make a distribution

In [ ]:
pca_gabor_benign = pca_vectorizer_b.transform(gabor_benign)
pca_gabor_malignant = pca_vectorizer_m.transform(gabor_malignant)

In [ ]:
pca_gabor_benign.shape, gabor_malignant.shape

In [ ]:
def print_gaussian(pca_gabor_benign, pca_gabor_malignant):
    for component in range(10):
        mean_benign = pca_gabor_benign[:, component].mean()
        std_benign = pca_gabor_benign[:, component].std()
        
        mean_malignant = pca_gabor_malignant[:, component].mean()
        std_malignant = pca_gabor_malignant[:, component].std()
        
        plt.plot(x, gaussian_dist(x, mean_benign, std_benign), label=f"Benign-component-{component}")
        plt.plot(x, gaussian_dist(x, mean_malignant, std_malignant), label=f"Malignant-{component}")

    plt.legend()
    plt.show()

In [ ]:
print_gaussian(pca_gabor_benign, pca_gabor_malignant)